Definitions: [Nguyen & O'Connor - Posterior calibration and exploratory analysis for natural lanaguge processing models](https://arxiv.org/pdf/1508.05154.pdf)

## Goal of Calibration

Suppose you have a probability / confidence score of a particular binary label. Let's call this $q, 0 \leq q \leq 1$. In the best case (i.e. a perfectly calibrated model), every time the model outputs a confidence score of $q$ the label is $1$ $q\%$ of the time. Since confidence scores can exist on a continuous scale, it's unlikely that the model will output a confidence score of exactly $q$ every time. To measure calibartion error then, we can use adaptive binning. That is, we take a range of confidence scores (e.g. $40-50\%$). If the model outputs confidence scores fairly uniformly, we can expect that all of the predictions with confidence scores in the range of $40-50\%$ will average $45\%$. But if $100\%$ of the labels are 0, then the model is very poorly calibrated. The model might still perfectly accurate, but having a confidence doesn't tell us anything. Then calibration error is the root of the mean square error between the empirical and predicted probabilities.

The calibration error is :

$$\sqrt{\frac{1}{N} \sum_{i=1}^T |B_i|(\hat{q}_i - \hat{p}_i)^2}$$

$\hat{p}_i$ is the average empirical label in bin $i$.

$$\hat{p} = \frac{1}{|B_i|} \sum_{k \in B_i} y_k$$


$\hat{q}_i$ is the average of all of the confidence scores in the bin.
$$\hat{q}_i = \frac{1}{|B_i|} \sum_{k \in B_i} q_k$$

### Contextualizing Calibration for CRFs

A CRF uses Viterbi to find the most likely labeling, but we need a confidence score. So, for a set of model predictions from a CRF, we need to find the marginal probability of each sequence using the constrained forward-backward algorithm, where the constraint is the entire sequence.

Then, we need true labels for each prediction

Calibration is binary and non-sequential in the Nguyen paper. We need to calibrate a model that outputs a sequence of labels, and these labels aren't binary.


Let's consider the non-sequential model, where we're predicting one label from a set $\ell, |\ell| > 2$. Then, we can calculate the calibration error multiple times, treating each label as the positive value. Then, we can average all of these.

Now, to make this useful for a sequential model, we can isolate the marginal probability for each label at each timestep. We can break out into supersenses (similar to N&O fig 4.)

For each single token in each sequence in streusle and for each label, get confidence score.

Use adaptive binning to measure confidence error. 

Example:

> The dog jumped over the cat.

Let's say there are 4 POS tags: D, N, V, P. Ground truth is `[D, N, V, P, D, N]`.

We need confidence scores for each label for each word.

This will be a 6 x 4 matrix, for the 6 words and 4 labels.

We now have 24 confidence scores in the range of 0 to 1. We can evenly divide the bins across the range of 0 - 1, or we can put every 24/num_bins elements into a bin sequentially.

If we want to calculate the calibration error for one label, we need that row containing 6 words. If we create 3 bins, we'll have two words in each bin. 

TODO: It would probably be better to have more items per bin (perhaps based on position in the sentence). Or bin multiple sample sentences instead of just one.

In [1]:
import math
import numpy as np

In [2]:
def single_label_calibration_error(confidence_scores, labels, num_bins):
    """Calculates calibration error using adaptive binning."""
    
    bins = []
    for i in range(num_bins):
        bins.append([])
    b = 0
    # TODO: handle last bin by merging last bin if it's not items_per_bin-sized
    items_per_bin = len(confidence_scores) / num_bins
    sorted_data = sorted(zip(confidence_scores, labels), key=lambda x: x[0])
    for (score, label) in sorted_data:
        if len(bins[b]) < items_per_bin:
            bins[b].append((score, label))
        else:
            b += 1
            bins[b].append((score, label))
    
    bin_square_errors = []
    
    for b in bins:
        average_score = sum([x for (x, y) in b]) / len(b)
        average_label = sum([y for (x, y) in b]) / len(b)
        square_error = (average_score - average_label) ** 2
        bin_square_errors.append(square_error)
        
    return math.sqrt(np.mean(bin_square_errors))

confidence_scores = [.10, .10, .30, .10, .10, .30]
labels = [1, 0, 0, 0, 1, 0]

single_label_calibration_error(confidence_scores, labels, 3)

0.36968455021364727

In [3]:
# TODO: Combine CE over all labels
# TODO: Investigate other methods of labeling calibration error (e.e. max)